In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import scipy.stats
import collections
import seaborn
import statsmodels.stats.proportion
import matplotlib

seaborn.set_style("darkgrid")
plt.rcParams.update({"font.size": 24})

In [2]:
raw_df = pd.read_csv("raw_Apr24.csv", index_col=None)

In [3]:
raw_df.head()

,Type,Url,Filename,MimeType,CreativeID,Media,Category,Brand,Product,Country,...,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,masculine_faces,feminine_faces
0,Creative,https://adintel.portfolio.intl-media.nielsen.c...,1012955614_1.mpg,video/mp4,1012955614,Television,"['COSMETICS & TOILETRIES', 'Cosmetics', 'Eye M...",Maybelline,Lash Sensational Sky High mascara,USA,...,0.0,53.0,150.0,14.0,4.0,0.0,0.0,0.0,8.0,213.0
1,Creative,https://adintel.portfolio.intl-media.nielsen.c...,1096386414_1.mpg,video/mp4,1096386414,Television,"['COSMETICS & TOILETRIES', 'Skincare Female', ...",CeraVe,female body moisturisers range,USA,...,0.0,0.0,24.0,54.0,92.0,29.0,0.0,0.0,115.0,84.0
2,Creative,https://adintel.portfolio.intl-media.nielsen.c...,1135302114_1.mpg,video/mp4,1135302114,Television,"['COSMETICS & TOILETRIES', 'Skincare Female', ...",CeraVe,female body moisturisers range,USA,...,0.0,64.0,211.0,72.0,18.0,26.0,0.0,0.0,215.0,176.0
3,Creative,https://adintel.portfolio.intl-media.nielsen.c...,1195194214_1.mp4,video/mpeg,1195194214,Internet,"['COSMETICS & TOILETRIES', 'Haircare', 'Hairca...",L'Oreal,haircare corporate,Italy,...,5.0,351.0,338.0,34.0,8.0,10.0,0.0,0.0,39.0,707.0
4,Creative,https://adintel.portfolio.intl-media.nielsen.c...,1284028214_1.mp4,video/mpeg,1284028214,Internet,"['COSMETICS & TOILETRIES', 'Cosmetics', 'Lipst...",Maybelline,Color Sensational Ultimate lipstick,Germany,...,5.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0


In [8]:
raw_df.columns[:30]

Index(['Type', 'Url', 'Filename', 'MimeType', 'CreativeID', 'Media',
       'Category', 'Brand', 'Product', 'Country', 'Title', 'TransmissionDate',
       'InitialChannel', 'Urls', 'Length', 'Version', 'LastRunDate',
       'Advertiser', 'AdType', 'Description', 'EndLine', 'cnsPaths',
       'frames_analyzed', '(female, [0, 15), [-inf, 1.1))',
       '(female, [0, 15), [1.1, 2.1))', '(female, [0, 15), [2.1, 3.1))',
       '(female, [0, 15), [3.1, 4.1))', '(female, [0, 15), [4.1, 5.1))',
       '(female, [0, 15), [5.1, 6.1))', '(female, [0, 15), [6.1, 7.1))'],
      dtype='object')

In [9]:
raw_df.shape

(1957, 146)

In [10]:
collections.Counter(raw_df["Type"])

Counter({'Creative': 1957})

In [11]:
collections.Counter(raw_df["MimeType"])

Counter({'video/mp4': 891, 'video/mpeg': 1066})

In [12]:
collections.Counter(raw_df["Media"])

Counter({'Television': 891, 'Internet': 1066})

In [13]:
collections.Counter(raw_df["Category"])

Counter({"['COSMETICS & TOILETRIES', 'Cosmetics', 'Eye Make Up']": 163,
         "['COSMETICS & TOILETRIES', 'Skincare Female', 'Female Body Moisturiser']": 2,
         "['COSMETICS & TOILETRIES', 'Haircare', 'Haircare Range/Corporate']": 326,
         "['COSMETICS & TOILETRIES', 'Cosmetics', 'Lipstick/gloss']": 73,
         "['COSMETICS & TOILETRIES', 'Cosmetics', 'Face/Foundation']": 111,
         "['COSMETICS & TOILETRIES', 'Skincare Female', 'Female Face Moisturiser Antiage/Antistain']": 240,
         "['COSMETICS & TOILETRIES', 'Haircare', 'Colourants']": 190,
         "['COSMETICS & TOILETRIES', 'Toiletries Range', 'Toiletries Range']": 256,
         "['COSMETICS & TOILETRIES', 'Skincare Female', 'Female Face Cleanser General/Wipes']": 89,
         "['COSMETICS & TOILETRIES', 'Suncare', 'Sun Protection']": 41,
         "['COSMETICS & TOILETRIES', 'Skincare Female', 'Female Facecare Range (Cleanser & Moisturiser)']": 27,
         "['COSMETICS & TOILETRIES', 'Skincare Unisex', 'Acn

In [27]:
categories = []

for categories_str in raw_df["Category"]:
    for category_str in categories_str[1:-1].split(","):
        category = category_str.replace("'", "").strip()
        categories.append(category)

category_dist = collections.Counter(categories)
category_tups = sorted(category_dist.items(), key=lambda tup: tup[1], reverse=True)
for cat, count in category_tups:
    print(cat, "\t", count)

COSMETICS & TOILETRIES 	 1957
Haircare 	 760
Toiletries Range 	 512
Skincare Female 	 452
Cosmetics 	 372
Haircare Range/Corporate 	 326
Female Face Moisturiser Antiage/Antistain 	 240
Colourants 	 190
Eye Make Up 	 163
Shampoo Regular 	 122
Face/Foundation 	 111
Female Face Cleanser General/Wipes 	 89
Lipstick/gloss 	 73
Hair Treatment - Oil/Cream/Mousse/Spray 	 68
Suncare 	 43
Sun Protection 	 41
Female Face Masks 	 40
Female Face Moisturiser General 	 34
Skincare Unisex 	 33
Hair Wash Range 	 31
Female Facecare Range (Cleanser & Moisturiser) 	 27
Acne/Antibacterial Face Treatments 	 25
Makeup Range 	 25
Deodorants & Anti-Perspirants 	 20
Skincare Male 	 19
Male Deodorants & Anti-Perspirants 	 16
Male Face Moisturiser General/Facecare Range 	 14
Conditioner 	 12
Female Face Cleanser Anti-Ageing 	 6
Female Face Cleanser Range 	 6
Unisex Facecare 	 6
Female Skincare Range (Face & Body) 	 5
Female Deodorants & Anti-Perspirants 	 4
Shampoo Anti-Dandruff/Anti-Fungal 	 4
Hair Treatment - R

In [23]:
collections.Counter(raw_df["Brand"])

Counter({'Maybelline': 204,
         'CeraVe': 8,
         "L'Oreal": 944,
         'Garnier': 652,
         'Kerastase': 114,
         "L'Oreal Professionnel": 28,
         "L'Oreal/Garnier": 4,
         'Maybelline Jade': 1,
         'Maybelline/Garnier': 2})

In [24]:
collections.Counter(raw_df["Product"])

Counter({'Lash Sensational Sky High mascara': 21,
         'female body moisturisers range': 2,
         'haircare corporate': 17,
         'Color Sensational Ultimate lipstick': 3,
         'SuperStay Matte Ink lipstick': 41,
         'Volume Million Lashes mascara': 5,
         'eye makeup range': 8,
         'Bambi Eye mascara range': 5,
         'Instant Age Rewind Eraser concealer': 2,
         'Sky High Curves mascara': 10,
         'Revitalift Crystal Fresh Hydrating female antiage face moisturiser': 3,
         'SuperStay 30H foundation': 8,
         'Preference hair colourant range': 10,
         'toiletries corporate': 212,
         'toiletries range': 20,
         'Micellar Rose micellar water': 7,
         'Revitalift Derm Intensives Night Serum female anti-age face moisturiser': 1,
         'UV Defender sun protection': 18,
         'Age Perfect female facecare range': 9,
         'Bright Complete acne face treatment': 8,
         'Preference hair colourant': 6,
         '

In [25]:
collections.Counter(raw_df["Country"])

Counter({'USA': 138,
         'Italy': 407,
         'Germany': 614,
         'Indonesia': 137,
         'UK': 152,
         'France': 176,
         'Spain': 216,
         'Mexico': 53,
         'Brazil': 64})

In [26]:
collections.Counter(raw_df["Advertiser"])

Counter({"L'Oreal": 1957})

## Read Data

In [2]:
# read data

raw_df = pd.read_csv("raw_Mar31.csv", index_col=None)
for i, column in enumerate(raw_df.columns):
    end = "\n" if i % 5 == 0 and i > 0 else "\t"
    print(f"{column:30s}", end=end)
print()
print(f"total number of rows = {raw_df.shape[0]}")
n_videos = raw_df["Filename"].str.endswith(".mp4").sum()
n_videos_rp = ((raw_df != -1).all(axis=1) & raw_df["Filename"].str.endswith(".mp4")).sum()
print(f"total number of videos = {n_videos}")
print(f"total number of videos with RP output = {n_videos_rp}")

Filename                      	Advertiser                    	Date                          	Version                       	Country                       	Duration                      
Description                   	Brand/Product                 	ProductAdvertiser             	Title                         	Creative ID                   
Channel                       	Publication                   	Website                       	!!MISSING_KEY!! CreativeLabel_Outlet_PaidSocial	Ad Format                     
Endline                       	video_key                     	frames_analyzed               	(female, [0, 15), [-inf, 1.1))	(female, [0, 15), [1.1, 2.1)) 
(female, [0, 15), [2.1, 3.1)) 	(female, [0, 15), [3.1, 4.1)) 	(female, [0, 15), [4.1, 5.1)) 	(female, [0, 15), [5.1, 6.1)) 	(female, [0, 15), [6.1, 7.1)) 
(female, [0, 15), [7.1, 8.1)) 	(female, [0, 15), [8.1, 9.1)) 	(female, [15, 25), [-inf, 1.1))	(female, [15, 25), [1.1, 2.1))	(female, [15, 25), [2.1, 3.1))
(female, [15, 25), [3

## Filter Data

In [3]:
# filter data

data_df = raw_df.loc[(raw_df != -1).all(axis=1) & raw_df["Filename"].str.endswith(".mp4")].copy()
data_df.index = pd.RangeIndex(0, len(data_df))
print(data_df.shape)

(7648, 148)


## Check Data

In [4]:
# check cross section bins count equals aggregates

def describe(row):
    print(row["Creative ID"])
    rp_columns = [column for column in row.index if re.match(r"\((fe)?male", column)]
    for column in rp_columns:
        if row[column] != 0:
            print(f"\t{column} : {row[column]}")
    print(f"faces : {row.faces}")
    print(f"gender : male = {row.masculine_faces}, female = {row.feminine_faces}")
    print("age : ")
    for j in range(7):
        count = row[f"age_{j + 1}"]
        end = ", " if j < 6 else "\n"
        print(f"age_{j + 1} = {count}", end=end)
    print("monk skin tone : ")
    for k in range(10):
        count = row[f"mst_scale_{k + 1}"]
        end = ", " if k < 9 else "\n"
        print(f"mst_scale_{k + 1} = {count}", end=end)

for _, row in data_df.iterrows():
    rp_data = np.zeros((2, 7, 10), dtype=int)
    for i, gender in enumerate(["male", "female"]):
        for j in range(7):
            for k in range(10):
                age_lb = 0 if j == 0 else 10 * j + 5
                age_ub = 10 * j + 15
                mst_lb = "-inf" if k == 0 else k + 0.1
                mst_ub = k + 1.1
                column_name = f"({gender}, [{age_lb}, {age_ub}), [{mst_lb}, {mst_ub}))"
                if column_name in row:
                    rp_data[i, j, k] = row[column_name]
    assert rp_data.sum() == row["faces"]
    assert rp_data[0].sum() == row["masculine_faces"]
    assert rp_data[1].sum() == row["feminine_faces"]
    for j in range(7):
        assert rp_data[:,j].sum() == row[f"age_{j + 1}"]
    # for k in range(10):
    #     assert rp_data[:, :, k].sum() == row[f"mst_scale_{k + 1}"]

## Metadata Distributions

In [5]:
# metadata

data_df["country"] = data_df["Country"]
data_df["brand"] = data_df["ProductAdvertiser"]
data_df["product type"] = data_df["brand_category"]
# data_df.loc[data_df["Brand/Product"].apply(lambda x: re.search("(\s|^)((toiletries)|(toilet))(\s|$)", x) is not None), 
#             "product type"] = "toiletries"

def plural(metadata):
    if metadata == "country":
        return "countries"
    elif metadata == "brand":
        return "brands"
    elif metadata == "product type":
        return "product types"
    elif metadata == "product target gender":
        return "product target gender categories"
    else:
        return metadata + "s"

def target_gender(desc):
    desc = str(desc)
    f = re.search(r"(^|\s)(females?)|(wom(a|e)n)(\s|$)", desc.lower()) is not None
    m = re.search(r"(^|\s)((?<!fe)males?)|((?<!wo)m(a|e)n)(\s|$)", desc.lower()) is not None
    u = re.search(r"(^|\s)unisex(\s|$)", desc.lower()) is not None
    if f and not m and not u:
        return "female"
    elif m and not f:
        return "male"
    elif u and not f and not m:
        return "unisex"
    else:
        return "unspecified"

def plot_metadata_distribution(dist, metadata):
    plt.figure(figsize=(22, 10))
    x = [item[0] for item in dist]
    y = [item[1] for item in dist]
    total = sum(y)
    yper = [100*yy/total for yy in y]
    xx = np.arange(len(x))
    bc = plt.bar(xx, y, width=0.8, edgecolor="black", linewidth=2)
    plt.bar_label(bc, [f"{yy:.1f}%" for yy in yper], padding=10, rotation=0)
    bottom, top = plt.ylim()
    plt.ylim(bottom, top + 50)
    xrotation = 90 if len(x) > 12 else 0
    plt.xticks(xx, x, rotation=xrotation, ha="center", fontsize="small")
    plt.yticks(fontsize="small")
    plt.xlabel(metadata.upper())
    plt.ylabel("NUMBER OF ADS")
    metadata_in_title = plural(metadata)
    plt.title(f"DISTRIBUTION OF ADS ACROSS {metadata_in_title.upper()}")
    plt.savefig(f"plots/metadata/{metadata}.png", bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

country_dist = sorted(collections.Counter(data_df["country"]).items(), key=lambda x: x[1], reverse=True)
brand_dist = sorted(collections.Counter(data_df["brand"]).items(), key=lambda x: x[1], reverse=True)
producttype_dist = sorted(collections.Counter(data_df["product type"]).items(), key=lambda x: x[1], reverse=True)
data_df["product target gender"] = data_df["Brand/Product"].apply(target_gender)
targetgender_dist = sorted(collections.Counter(data_df["product target gender"]).items(), key=lambda x: x[1], 
                           reverse=True)
total = len(data_df)

print("country")
for i, (country, count) in enumerate(country_dist):
    per = 100*count/total
    end = "\n" if i % 5 == 0 and i > 0 else "\t"
    print(f"{country:10s} : {count:4d} ({per:4.1f}%)", end=end)
plot_metadata_distribution(country_dist, "country")
print("\n")

print("brand")
for i, (brand, count) in enumerate(brand_dist):
    per = 100*count/total
    end = "\n" if i % 5 == 0 and i > 0 else "\t"
    print(f"{brand:10s} : {count:4d} ({per:4.1f}%)", end=end)
plot_metadata_distribution(brand_dist, "brand")
print("\n")

print("product type")
for i, (producttype, count) in enumerate(producttype_dist):
    per = 100*count/total
    end = "\n" if i % 5 == 0 and i > 0 else "\t"
    print(f"{producttype:10s} : {count:4d} ({per:4.1f}%)", end=end)
plot_metadata_distribution(producttype_dist, "product type")
print("\n")

print("product target gender")
for i, (targetgender, count) in enumerate(targetgender_dist):
    per = 100*count/total
    end = "\n" if i % 5 == 0 and i > 0 else "\t"
    print(f"{targetgender:10s} : {count:4d} ({per:4.1f}%)", end=end)
plot_metadata_distribution(targetgender_dist, "product target gender")

country
Germany    : 3202 (41.9%)	UK         : 1715 (22.4%)	France     :  817 (10.7%)	Italy      :  806 (10.5%)	USA        :  701 ( 9.2%)	Spain      :  227 ( 3.0%)
Brazil     :   54 ( 0.7%)	India      :   49 ( 0.6%)	Mexico     :   46 ( 0.6%)	Indonesia  :   30 ( 0.4%)	Canada     :    1 ( 0.0%)


brand
GARNIER    : 1825 (23.9%)	OAP        :  880 (11.5%)	LRP        :  631 ( 8.3%)	YSL        :  613 ( 8.0%)	VICHY      :  511 ( 6.7%)	LANCOME    :  504 ( 6.6%)
KIEHLS     :  429 ( 5.6%)	MNY        :  408 ( 5.3%)	MAYBELLINE :  346 ( 4.5%)	ARMANI     :  337 ( 4.4%)	KERASTASE  :  335 ( 4.4%)
NYX        :  286 ( 3.7%)	CERAVE     :  239 ( 3.1%)	LOP        :  110 ( 1.4%)	PRADA      :   84 ( 1.1%)	BIOTHERM   :   80 ( 1.0%)
REDKEN     :   30 ( 0.4%)	

product type
makeup     : 2897 (37.9%)	skincare   : 2429 (31.8%)	haircare   : 1640 (21.4%)	fragrance  :  682 ( 8.9%)	

product target gender
unspecified : 4844 (63.3%)	female     : 2155 (28.2%)	male       :  460 ( 6.0%)	unisex     :  189 ( 2.5%)	

In [6]:
def metadata_x_metadata_stacked(data_df, primary_metadata, secondary_metadata, figsize=(22, 10)):
    assert primary_metadata in ["brand", "country", "product type", "product target gender"]
    assert secondary_metadata in ["brand", "country", "product type", "product target gender"]
    n_primary_metadata_cats = data_df[primary_metadata].unique().size
    primary_metadata_dist = collections.Counter(data_df[primary_metadata].tolist())
    primary_metadata_cats = sorted(primary_metadata_dist.keys(), key=lambda cat: primary_metadata_dist[cat], 
                                   reverse=True)
    n_secondary_metadata_cats = data_df[secondary_metadata].unique().size
    secondary_metadata_dist = collections.Counter(data_df[secondary_metadata].tolist())
    secondary_metadata_cats = sorted(secondary_metadata_dist.keys(), key=lambda cat: secondary_metadata_dist[cat], 
                                   reverse=True)
    metadata_x_metadata_arr = np.zeros((n_primary_metadata_cats, n_secondary_metadata_cats), dtype=int)
    for (primary_metadata_cat, secondary_metadata_cat), df in data_df.groupby([primary_metadata, secondary_metadata]):
        i = primary_metadata_cats.index(primary_metadata_cat)
        j = secondary_metadata_cats.index(secondary_metadata_cat)
        metadata_x_metadata_arr[i, j] = len(df)
    metadata_x_metadata_arr = 100 * metadata_x_metadata_arr / metadata_x_metadata_arr.sum(axis=1, keepdims=True)
    bar_width = 0.8
    bottom = np.zeros(n_primary_metadata_cats, dtype=float)
    x = np.arange(n_primary_metadata_cats)
    plt.figure(figsize=figsize)
    for j, secondary_metadata_cat in enumerate(secondary_metadata_cats):
        height = metadata_x_metadata_arr[:, j]
        plt.bar(x, height, width=bar_width, bottom=bottom, label=secondary_metadata_cat, edgecolor="black", lw=2)
        bottom += height
    plt.legend(bbox_to_anchor=(1, 1))
    xrotation = 90 if n_primary_metadata_cats > 12 else 0
    plt.xticks(x, primary_metadata_cats, fontsize="small", rotation=xrotation)
    plt.xlabel(primary_metadata)
    plt.ylabel("PERCENTAGE OF ADS")
    plt.title(f"{secondary_metadata.upper()} DISTRIBUTION OF {primary_metadata.upper()} ADS")
    plt.savefig(f"plots/metadata_x_metadata/{primary_metadata}_x_{secondary_metadata}.png", bbox_inches="tight", 
                pad_inches=0, dpi=300)
    plt.close()

In [15]:
metadata_x_metadata_stacked(data_df, "brand", "product type")
metadata_x_metadata_stacked(data_df, "country", "product type")
metadata_x_metadata_stacked(data_df, "brand", "product target gender")
metadata_x_metadata_stacked(data_df, "country", "product target gender")
metadata_x_metadata_stacked(data_df, "product type", "product target gender")
metadata_x_metadata_stacked(data_df, "brand", "country", figsize=(22, 15))
metadata_x_metadata_stacked(data_df, "country", "brand", figsize=(22, 15))

In [20]:
df = data_df.loc[data_df["brand"] == "PRADA"].groupby("product target gender").count()["Creative ID"]
(100*df/df.sum()).round(2)

product target gender
female         26.19
male           59.52
unspecified    14.29
Name: Creative ID, dtype: float64

In [21]:
df = data_df.loc[data_df["product type"] == "fragrance"].groupby("product target gender").count()["Creative ID"]
(100*df/df.sum()).round(2)

product target gender
female         62.61
male           30.50
unspecified     6.89
Name: Creative ID, dtype: float64

## Create Data

In [8]:
# rp_data rows x gender x age x skin tone

gender_cats = ["male", "female"]
age_cats = [(0, 15), (15, 25), (25, 35), (35, 45), (45, 55), (55, 65), (65, 75)]
age_cats2 = ["<15y", "15y-25y", "25y-35y", "35y-45y", "45-75y"]
skintone_cats = [(-np.inf, 1.1), (1.1, 2.1), (2.1, 3.1), (3.1, 4.1), (4.1, 5.1),
                 (5.1, 6.1), (6.1, 7.1), (7.1, 8.1), (8.1, 9.1), (9.1, 10.1)]
skintone_cats2 = ["light", "medium", "dark"]

rp_data = np.zeros((len(data_df), len(gender_cats), len(age_cats), len(skintone_cats)), dtype=int)

for i, row in data_df.iterrows():
    for j, gender in enumerate(gender_cats):
        for k, age_bin in enumerate(age_cats):
            age_lb, age_ub = age_bin
            for l, skintone_bin in enumerate(skintone_cats):
                skintone_lb, skintone_ub = skintone_bin
                column = f"({gender}, [{age_lb}, {age_ub}), [{skintone_lb}, {skintone_ub}))"
                if column in row.index:
                    rp_data[i, j, k, l] += row[column]

rp_data2 = np.zeros((len(data_df), len(gender_cats), len(age_cats2), len(skintone_cats2)), dtype=int)
rp_data2[:, :, :4, 0] = rp_data[:, :, :4, :3].sum(axis=3)
rp_data2[:, :, :4, 1] = rp_data[:, :, :4, 3:7].sum(axis=3)
rp_data2[:, :, :4, 2] = rp_data[:, :, :4, 7:].sum(axis=3)
rp_data2[:, :, 4, 0] = rp_data[:, :, 4:, :3].sum(axis=(2,3))
rp_data2[:, :, 4, 1] = rp_data[:, :, 4:, 3:7].sum(axis=(2, 3))
rp_data2[:, :, 4, 2] = rp_data[:, :, 4:, 7:].sum(axis=(2,3))

## Demographic Identity vs Metadata

In [9]:
# one metadata vs one rp

def metadata_x_rp(df, rp_data, metadata, rp, confidence=0.05, condition=None):
    assert rp in ["gender", "age", "skin tone"]
    assert metadata in ["country", "brand", "product type", "product target gender"]
    
    if condition is None:
        plot_filename = f"plots/rp_x_metadata/{rp}_x_{metadata}.png"
        text_filename = f"text/rp_x_metadata/{rp}_x_{metadata}.txt"
    else:
        plot_filename = f"plots/rp_x_metadata_x_metadata/{rp}_x_{metadata}_{condition}.png"
        text_filename = f"text/rp_x_metadata_x_metadata/{rp}_x_{metadata}_{condition}.txt"
    
    metadata_cat_to_count = collections.Counter(df[metadata])
    metadata_cats = [metadata_cat for metadata_cat, count in metadata_cat_to_count.items() if count >= 30]
    removed_metadata_cats = [x for x in df[metadata].unique() if x not in metadata_cats]
    if len(metadata_cats) <= 1:
        return
    
    title = f"{rp} vs {metadata}"
    if condition is not None:
        title += f" {condition}"
    
    
    print(title)
    print(f"{metadata} categories : {metadata_cats}")
    print(f"removed categories : {removed_metadata_cats}")
    
    fw = open(text_filename, "w")
    fw.write(f"{title}\n")
    fw.write(f"{metadata} cats = {metadata_cats}\n")
    fw.write(f"removed {metadata} cats = {removed_metadata_cats}\n")
    
    if rp == "gender":
        rp_cats = gender_cats
    elif rp == "age":
        rp_cats = age_cats2
    else:
        rp_cats = skintone_cats2
    
    rp_arr = np.zeros((df[metadata].isin(metadata_cats).sum(), len(rp_cats)))
    metadata_arr = []
    j = 0
    
    for i, row in df.iterrows():
        metadata_value = row[metadata]
        if metadata_value in metadata_cats:
            metadata_arr.append(metadata_value)
            if rp == "gender":
                rp_arr[j] = 100 * rp_data[i].sum(axis=(1, 2))/rp_data[i].sum()
            elif rp == "age":
                rp_arr[j] = 100 * rp_data[i].sum(axis=(0, 2))/rp_data[i].sum()
            else:
                rp_arr[j] = 100 * rp_data[i].sum(axis=(0, 1))/rp_data[i].sum()
            j += 1
    
    metadata_arr = np.array(metadata_arr)
    most_common_metadata_cat = rp_arr.sum(axis=0).argmax()
    metadata_cats = sorted(metadata_cats, key=lambda x: rp_arr[:, most_common_metadata_cat][metadata_arr == x].mean(), 
                           reverse=True)
    
    plt.figure(figsize=(22, 10))
    x = np.arange(len(metadata_cats))
    group_width = 0.8
    bar_width = group_width/len(rp_cats)

    for i, rp_cat in enumerate(rp_cats):
        rp_cat_arr = rp_arr[:, i]
        y = [rp_cat_arr[metadata_arr == metadata_cat].mean() for metadata_cat in metadata_cats]
        bc = plt.bar(x + i * bar_width, y, width=bar_width, label=str(rp_cat), edgecolor="black", linewidth=2)
        plt.bar_label(bc, [f"{yy:.1f}%" for yy in y], padding=5, rotation=90, fontsize="small")
    
    bottom, top = plt.ylim()
    plt.ylim(bottom, top + 10)
    plt.ylabel("AVERAGE SCREEN TIME")
    xrotation = 90 if len(x) > 10 else 0
    plt.xticks(x + bar_width*(len(rp_cats) - 1)/2, metadata_cats, ha="center", rotation=xrotation)
    plt.yticks(fontsize="small")
    plt.xlabel(metadata.upper())
    condition_title = " (" + condition + ")" if condition is not None else ""
    plt.title(f"DISTRIBUTION OF {rp.upper()} SCREEN TIME ACROSS {plural(metadata).upper()}{condition_title}")
    plt.legend()
    plt.savefig(plot_filename, bbox_inches="tight", pad_inches=0, dpi=300)
    plt.close()

    for i, rp_cat in enumerate(rp_cats):
        rp_cat_arr = rp_arr[:, i]
        metadata_to_rp_arr = {metadata_cat:rp_cat_arr[metadata_arr == metadata_cat] for metadata_cat in metadata_cats}
        metadata_sorted_by_mean = sorted(metadata_cats, key=lambda x: metadata_to_rp_arr[x].mean(), reverse=True)
        pvalues = []
        
        for j in range(len(metadata_cats) - 1):
            cat1, cat2 = metadata_sorted_by_mean[j], metadata_sorted_by_mean[j + 1]
            arr1, arr2 = metadata_to_rp_arr[cat1], metadata_to_rp_arr[cat2]
            _, pvalue = scipy.stats.ttest_ind(arr1, arr2, equal_var=False, alternative="greater")
            pvalues.append(pvalue)
        significant = np.array(pvalues)/(len(metadata_cats) - 1) < confidence
        print(f"{rp_cat} : ", end="")
        fw.write(f"{rp_cat}\n")
        
        for j in range(len(metadata_cats) - 1):
            cat = metadata_sorted_by_mean[j]
            sign = ">" if significant[j] else "="
            print(f"{cat} {sign} ", end="")
            fw.write(f"{cat} {sign} ")
        print(metadata_sorted_by_mean[-1])
        fw.write(f"{metadata_sorted_by_mean[-1]}\n")
    print()
    fw.write("\n")
    fw.close()

In [10]:
metadata_vars = ["country", "brand", "product type", "product target gender"]
rp_vars = ["gender", "age", "skin tone"]

for metadata in metadata_vars:
    for rp in rp_vars:
        metadata_x_rp(data_df, rp_data2, metadata, rp)

gender vs country
country categories : ['Mexico', 'Brazil', 'USA', 'France', 'Germany', 'Italy', 'Spain', 'UK', 'India', 'Indonesia']
removed categories : ['Canada']
male : India > UK > France > Germany > USA > Italy > Brazil > Mexico = Spain > Indonesia
female : Indonesia > Spain = Mexico > Brazil > Italy > USA > Germany > France > UK > India

age vs country
country categories : ['Mexico', 'Brazil', 'USA', 'France', 'Germany', 'Italy', 'Spain', 'UK', 'India', 'Indonesia']
removed categories : ['Canada']
<15y : Germany > USA > UK = Italy > Spain > France > Indonesia > Brazil > Mexico > India
15y-25y : Brazil > USA > Germany > France > UK > Italy > Indonesia = Spain > Mexico > India
25y-35y : India > Indonesia > Mexico > Spain > Italy > France > UK > Germany = Brazil > USA
35y-45y : Mexico > India > UK > Spain > France > Germany > Italy = USA > Brazil > Indonesia
45-75y : Germany > UK = France > Spain = Italy > Mexico > USA > India > Brazil > Indonesia

skin tone vs country
country cate

## Demographic Identity Diversity vs Metadata

In [8]:
# metadata vs rp var

def metadata_x_rp_std(df, rp_data, metadata, rp, confidence=0.05):
    assert rp in ["gender", "age", "skin tone"]
    assert metadata in ["country", "advertiser", "product type", "target gender"]
    if metadata == "country":
        column = "Country"
    elif metadata == "advertiser":
        column = "ProductAdvertiser"
    elif metadata == "product type":
        column = "brand_category"
    else:
        column = "targetgender"
    metadata_cat_to_count = collections.Counter(df[column])
    metadata_cats = [metadata_cat for metadata_cat, count in metadata_cat_to_count.items() if count >= 30]
    metadata_cats = sorted(metadata_cats, key=lambda x: metadata_cat_to_count[x], reverse=True)
    removed_metadata_cats = [x for x in df[column].unique() if x not in metadata_cats]
    print(f"{metadata} : {metadata_cats}")
    print(f"removed columns : {removed_metadata_cats}")
    if rp == "gender":
        rp_cats = gender_cats
    elif rp == "age":
        rp_cats = age_cats2
    else:
        rp_cats = skintone_cats2
    rp_arr = np.zeros((df[column].isin(metadata_cats).sum(), len(rp_cats)))
    metadata_arr = []

    j = 0
    for i, row in df.iterrows():
        metadata_value = row[column]
        if metadata_value in metadata_cats:
            metadata_arr.append(metadata_value)
            if rp == "gender":
                rp_arr[j] = rp_data[i].sum(axis=(1, 2))
            elif rp == "age":
                rp_arr[j] = rp_data[i].sum(axis=(0, 2))
            else:
                rp_arr[j] = rp_data[i].sum(axis=(0, 1))
            j += 1
    metadata_arr = np.array(metadata_arr)

    plt.figure(figsize=(25, 10))
    colors = plt.get_cmap("Paired").colors
    x = np.arange(len(metadata_cats))
    bar_width = 0.8
    rv = 2 * np.arange(len(rp_cats)).reshape(1, -1)/(len(rp_cats) - 1)
    n = rp_arr.sum(axis=1)
    mean = (rp_arr * rv).sum(axis=1)/n
    var = ((rp_arr * rv**2).sum(axis=1) - n * mean**2)/(n - 1 + 1e-8)
    stddev = np.sqrt(var)
    y = [stddev[metadata_arr == metadata_cat].mean() for metadata_cat in metadata_cats]
    metadata_cats = np.array(metadata_cats)
    metadata_cats = metadata_cats[np.argsort(y)]
    y = 100*np.sort(y)
    bc = plt.bar(x, y, width=bar_width, color=colors[2])
    plt.bar_label(bc, [f"{yy:.1f}" for yy in y], padding=5)
    bottom, top = plt.ylim()
    plt.ylim(bottom, top + 10)
    plt.ylabel(f"100 * AVERAGE {rp.upper()} STDDEV")
    plt.xticks(x, metadata_cats, fontsize="small")
    plt.xlabel(metadata)
    plt.title(f"DISTRIBUTION OF AVERAGE {rp.upper()} STDDEV OVER {metadata.upper()}")
    plt.savefig(f"plots/{rp}_diversity_x_{metadata}.png")
    plt.close()

    metadata_to_rp_arr = {metadata_cat:stddev[metadata_arr == metadata_cat] for metadata_cat in metadata_cats}
    metadata_sorted_by_mean = sorted(metadata_cats, key=lambda x: metadata_to_rp_arr[x].mean())
    pvalues = []
    for j in range(len(metadata_cats) - 1):
        cat1, cat2 = metadata_sorted_by_mean[j], metadata_sorted_by_mean[j + 1]
        arr1, arr2 = metadata_to_rp_arr[cat1], metadata_to_rp_arr[cat2]
        _, pvalue = scipy.stats.ttest_ind(arr1, arr2, equal_var=False, alternative="less")
        pvalues.append(pvalue)
    significant = np.array(pvalues)/(len(metadata_cats) - 1) < confidence
    for j in range(len(metadata_cats) - 1):
        cat = metadata_sorted_by_mean[j]
        sign = "<" if significant[j] else "="
        print(f"{cat} {sign} ", end="")
    print(metadata_sorted_by_mean[-1])
    print()

In [9]:
metadata_vars = ["country", "advertiser", "product type", "target gender"]
rp_vars = ["gender", "age", "skin tone"]

for metadata in metadata_vars:
    for rp in rp_vars:
        metadata_x_rp_std(data_df, rp_data2, metadata, rp)

country : ['Germany', 'UK', 'France', 'Italy', 'USA', 'Spain', 'Brazil', 'India', 'Mexico', 'Indonesia']
removed columns : ['Canada']
Italy < Brazil < Spain < UK < Germany < France = Mexico < USA < India = Indonesia

country : ['Germany', 'UK', 'France', 'Italy', 'USA', 'Spain', 'Brazil', 'India', 'Mexico', 'Indonesia']
removed columns : ['Canada']
USA < Italy < Germany < UK < France < Spain < Brazil < India < Indonesia < Mexico

country : ['Germany', 'UK', 'France', 'Italy', 'USA', 'Spain', 'Brazil', 'India', 'Mexico', 'Indonesia']
removed columns : ['Canada']
India = Mexico < Indonesia < Germany < Spain < Italy < France < Brazil < USA < UK

advertiser : ['GARNIER', 'OAP', 'LRP', 'YSL', 'VICHY', 'LANCOME', 'KIEHLS', 'MNY', 'MAYBELLINE', 'ARMANI', 'KERASTASE', 'NYX', 'CERAVE', 'LOP', 'PRADA', 'BIOTHERM', 'REDKEN']
removed columns : []
REDKEN < PRADA < OAP < MNY < LANCOME < VICHY < MAYBELLINE < KERASTASE < BIOTHERM < CERAVE < GARNIER < ARMANI < LOP < KIEHLS < NYX < LRP < YSL

advertiser

## Demographic Identity vs Metadata Pair

In [11]:
metadata_columns = ["country", "brand", "product type", "product target gender"]

for i in range(len(metadata_columns)):
    for j in range(i + 1, len(metadata_columns)):
        col1 = metadata_columns[i]
        col2 = metadata_columns[j]
        gdf = data_df.groupby([col1, col2]).count()["Creative ID"].reset_index().rename(columns={"Creative ID": "n"})
        print(f"{col1} x {col2}")
        n = (gdf.n >= 30).sum()
        gdf1 = gdf[gdf.n >= 30].groupby(col1).count()["n"].reset_index()
        n1 = (gdf1.n > 1).sum()
        cats1 = gdf1.loc[gdf1.n > 1, col1].values
        gdf2 = gdf[gdf.n >= 30].groupby(col2).count()["n"].reset_index()
        n2 = (gdf2.n > 1).sum()
        cats2 = gdf2.loc[gdf2.n > 1, col2].values
        print(f"{n} ({col1}, {col2}) pairs have atleast 30 samples")
        print(f"{n1} {col1}s have atleast 2 {col2}s with atleast 30 samples: {cats1}")
        print(f"{n2} {col2}s have atleast 2 {col1}s with atleast 30 samples: {cats2}")
        print()

country x brand
52 (country, brand) pairs have atleast 30 samples
6 countrys have atleast 2 brands with atleast 30 samples: ['France' 'Germany' 'Italy' 'Spain' 'UK' 'USA']
13 brands have atleast 2 countrys with atleast 30 samples: ['ARMANI' 'CERAVE' 'GARNIER' 'KERASTASE' 'KIEHLS' 'LANCOME' 'LOP' 'LRP'
 'MAYBELLINE' 'NYX' 'OAP' 'VICHY' 'YSL']

country x product type
26 (country, product type) pairs have atleast 30 samples
6 countrys have atleast 2 product types with atleast 30 samples: ['France' 'Germany' 'Italy' 'Spain' 'UK' 'USA']
4 product types have atleast 2 countrys with atleast 30 samples: ['fragrance' 'haircare' 'makeup' 'skincare']

country x product target gender
20 (country, product target gender) pairs have atleast 30 samples
6 countrys have atleast 2 product target genders with atleast 30 samples: ['France' 'Germany' 'Italy' 'Spain' 'UK' 'USA']
4 product target genders have atleast 2 countrys with atleast 30 samples: ['female' 'male' 'unisex' 'unspecified']

brand x product

In [12]:
def metadata2_x_rp(df, rp_data, metadata1, metadata2, rp, confidence=0.05):
    assert rp in ["gender", "age", "skin tone"]
    assert metadata1 in ["country", "brand", "product type", "product target gender"]
    assert metadata2 in ["country", "brand", "product type", "product target gender"]
    metadata1_cats = df[metadata1].unique()
    metadata2_cats = df[metadata2].unique()
    
    for metadata1_cat in metadata1_cats:
        index = df[metadata1] == metadata1_cat
        fdf = df[index].copy()
        fdf.index = pd.RangeIndex(0, len(fdf))
        metadata_x_rp(fdf, rp_data[index], metadata=metadata2, rp=rp, confidence=confidence, 
                      condition=f"{metadata1}={metadata1_cat}")
    
    for metadata2_cat in metadata2_cats:
        index = df[metadata2] == metadata2_cat
        fdf = df[index].copy()
        fdf.index = pd.RangeIndex(0, len(fdf))
        metadata_x_rp(fdf, rp_data[index], metadata=metadata1, rp=rp, confidence=confidence, 
                      condition=f"{metadata2}={metadata2_cat}")

In [13]:
metadata_vars = ["country", "brand", "product type", "product target gender"]
rp_vars = ["gender", "age", "skin tone"]

for i in range(len(metadata_vars)):
    for j in range(i + 1, len(metadata_vars)):
        for rp in rp_vars:
            metadata2_x_rp(data_df, rp_data2, metadata_vars[i], metadata_vars[j], rp)

gender vs brand country=USA
brand categories : ['ARMANI', 'CERAVE', 'GARNIER', 'MAYBELLINE', 'NYX', 'OAP', 'YSL']
removed categories : ['KIEHLS', 'LANCOME', 'LRP', 'PRADA', 'REDKEN', 'VICHY']
male : CERAVE = YSL > OAP > ARMANI = MAYBELLINE > GARNIER > NYX
female : NYX > GARNIER > MAYBELLINE = ARMANI > OAP > YSL = CERAVE

gender vs brand country=France
brand categories : ['GARNIER', 'KIEHLS', 'LANCOME', 'LRP', 'MAYBELLINE', 'OAP', 'VICHY']
removed categories : ['ARMANI', 'BIOTHERM', 'CERAVE', 'KERASTASE', 'LOP', 'NYX', 'PRADA', 'YSL']
male : KIEHLS > LRP = VICHY > GARNIER > LANCOME > OAP > MAYBELLINE
female : MAYBELLINE > OAP > LANCOME > GARNIER > VICHY = LRP > KIEHLS

gender vs brand country=Germany
brand categories : ['ARMANI', 'BIOTHERM', 'CERAVE', 'GARNIER', 'KERASTASE', 'KIEHLS', 'LANCOME', 'LOP', 'LRP', 'MNY', 'NYX', 'VICHY', 'YSL']
removed categories : ['PRADA', 'REDKEN']
male : VICHY > LRP > ARMANI > NYX > YSL > CERAVE > KERASTASE > GARNIER > KIEHLS > BIOTHERM > LANCOME > MNY > 

In [22]:
arr = np.random.randint(0, 10, (4, 3))
arr

array([[4, 3, 7],
       [2, 0, 4],
       [5, 7, 9],
       [4, 9, 1]])

In [24]:
arr[[1, 2], [0, 2]]

array([2, 9])

In [26]:
arr[(arr >= 5).all(axis=1)]

array([[5, 7, 9]])

In [29]:
rp_data2[:, 1].sum(axis=(1, 2))

array([248,  36,   1, ..., 163, 167, 583])

In [30]:
rp_data2.sum(axis=(1, 2, 3))

array([291, 409, 214, ..., 164, 240, 640])

In [227]:
def gender_heatmap(df, rp_data, metadata1, metadata2, alpha=0.05):
    seaborn.set_style("dark")
    assert metadata1 in ["country", "brand", "product type", "product target gender"]
    assert metadata2 in ["country", "brand", "product type", "product target gender"]
    metadata1_cats_dist = collections.Counter(df[metadata1])
    metadata2_cats_dist = collections.Counter(df[metadata2])
    metadata1_cats = np.array(sorted(metadata1_cats_dist.keys(), key=lambda cat: metadata1_cats_dist[cat], 
                                     reverse=True))
    metadata2_cats = np.array(sorted(metadata2_cats_dist.keys(), key=lambda cat: metadata2_cats_dist[cat], 
                                     reverse=True))
    count_arr = np.zeros((len(metadata1_cats), len(metadata2_cats)), dtype=int)
    for (metadata1_cat, metadata2_cat), gdf in df.groupby([metadata1, metadata2]):
        i = (metadata1_cats == metadata1_cat).nonzero()[0].item()
        j = (metadata2_cats == metadata2_cat).nonzero()[0].item()
        count_arr[i, j] = len(gdf)
    row_index = (count_arr >= 30).any(axis=1)
    metadata1_cats, count_arr = metadata1_cats[row_index], count_arr[row_index]
    column_index = (count_arr >= 30).any(axis=0)
    metadata2_cats, count_arr = metadata2_cats[column_index], count_arr[:, column_index]
    value_arr = np.zeros((len(metadata1_cats), len(metadata2_cats)), dtype=float)
    for i, metadata1_cat in enumerate(metadata1_cats):
        for j, metadata2_cat in enumerate(metadata2_cats):
            ind = (df[metadata1] == metadata1_cat) & (df[metadata2] == metadata2_cat)
            if count_arr[i, j] < 30:
                value_arr[i, j] = np.nan
            else:
                n_female_faces = rp_data[ind, 1].sum(axis=(1, 2))
                n_total_faces = rp_data[ind].sum(axis=(1, 2, 3))
                value_arr[i, j] = (n_female_faces/n_total_faces).mean()
                res = scipy.stats.ttest_1samp(n_female_faces/n_total_faces, 0.5, alternative="two-sided")
                pvalue = res.pvalue
                if pvalue >= alpha:
                    value_arr[i, j] = np.nan
    cmap = matplotlib.colormaps["RdBu"](np.linspace(0.2, 0.9, 100))
    cmap = matplotlib.colors.ListedColormap(cmap)
    height = len(metadata1_cats) * 1.2
    width = len(metadata2_cats)
    plt.figure(figsize=(width, height))
    ax = seaborn.heatmap(100*value_arr, xticklabels=metadata2_cats, yticklabels=metadata1_cats, annot=True, fmt=".2f", 
                         linewidth=1, cmap=cmap, annot_kws={"fontsize":"xx-small"}, cbar=True, vmin=0, vmax=100, 
                         linecolor="black")
    ax.set_xticklabels(metadata2_cats, fontsize="xx-small")
    ax.set_yticklabels(metadata1_cats, fontsize="xx-small")
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize="xx-small")
    ax.set_xlabel(metadata2.upper(), fontsize="xx-small")
    ax.set_ylabel(metadata1.upper(), fontsize="xx-small")
    ax.set_title(f"Female Screentime for {metadata1.upper()} x {metadata2.upper()}", fontsize="xx-small")
    filename = f"plots/rp_x_metadata_x_metadata/gender_x_{metadata1}_x_{metadata2}.png"
    figure = ax.get_figure()
    figure.savefig(filename, dpi=300, bbox_inches="tight")
    plt.close()

In [228]:
gender_heatmap(data_df, rp_data2, "country", "brand")
gender_heatmap(data_df, rp_data2, "product type", "brand")
gender_heatmap(data_df, rp_data2, "product type", "country")
gender_heatmap(data_df, rp_data2, "product target gender", "brand")
gender_heatmap(data_df, rp_data2, "product target gender", "country")
gender_heatmap(data_df, rp_data2, "product target gender", "product type")